
# XGBoost_GridSearchCV_sinAuthoryGeometry

1. Vamos a cargar el dataset e importamos las librerias. 

2. Tras el análisis del EDA vamos a eliminiar las columnas "author" y "geometry". 

3. A continuación separamos el dataset en dos;
<BR>

i) Incluye los nulos (10.415) que será donde hagamos las prediciones.
<BR>

ii) No incluye los nulos (21.229) y será el que utilicemos para entrenar el modelo.

In [22]:
#Importamos las librerias, cargamos el fichero y vemos las columnas. 

import csv
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('data/data.csv')

data_copia = data.copy()

data_copia.shape

data_copia.head(2)

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.0,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2


In [2]:
# Crea un nuevo DataFrame eliminando las columnas "Author" y "geometría". 

new_df = data_copia.drop(["author", "geometry"], axis=1)

new_df.shape


(31644, 8)

In [23]:
# Separo el nuevo DataFrame en dos:

# Crear un DataFrame con filas que contienen NaN en 'x_e_out [-]'

df_nan = new_df[new_df['x_e_out [-]'].isnull()]

# Crear un DataFrame sin filas que contienen NaN en 'x_e_out [-]'

df_sin_nan = new_df[~new_df['x_e_out [-]'].isnull()]

print("Data_copia: ", new_df.shape)
print("Data con los NUN:", df_nan.shape)
print("Data sin los NUN: ", df_sin_nan.shape)

Data_copia:  (31644, 8)
Data con los NUN: (10415, 8)
Data sin los NUN:  (21229, 8)


# Probamos el modelo de XG_Boost y sacamos las principales metricas para ver el rendimiento. 


In [4]:
# Ahora vamos a entrenar el modelo 

import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


# Separar las características (X) y la variable objetivo (y)
X = df_sin_nan.drop('x_e_out [-]', axis=1)
y = df_sin_nan['x_e_out [-]']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto Scaler y ajustarlo a los datos de entrenamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar el modelo XGBoost con los mejores hiperparámetros
modelo_XGB = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100)
modelo_XGB.fit(X_train_scaled, y_train)


# Realizar predicciones en el conjunto de prueba
y_pred = modelo_XGB.predict(X_test_scaled)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Calcular el R cuadrado
r2 = r2_score(y_test, y_pred)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular el MAE
mae = mean_absolute_error(y_test, y_pred)

print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared:", r2)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

Root Mean Squared Error (RMSE): 0.07541543887865379
R-squared: 0.4400752308968243
Mean Squared Error (MSE): 0.005687488421259966
Mean Absolute Error (MAE): 0.05248833533336977


# Hacemos la predicción y pasamos a formato CVS para realizar la submission de Kaggle. 

In [26]:
import pandas as pd

# Preprocesar los datos del DataFrame 'df_nan'
X_pred = df_nan.drop('x_e_out [-]', axis=1)  # Eliminar la columna de la variable objetivo
X_pred_scaled = scaler.transform(X_pred)  # Escalar los datos de 'df_nan' usando el mismo Scaler utilizado en el entrenamiento

# Realizar las predicciones
y_pred = modelo_XGB.predict(X_pred_scaled)

# Crear un nuevo DataFrame con las predicciones
df_submission = pd.DataFrame({'ID': df_nan['id'], 'x_e_out [-]': y_pred})

# Guardar el DataFrame 'df_submission' como un archivo CSV para la presentación en Kaggle
df_submission.to_csv('submission13.csv', index=False)

df_submission.head(10)



,ID,x_e_out [-]
4,4,0.004058
7,7,-0.069166
10,10,-0.058018
12,12,-0.000448
23,23,0.047946
27,27,-0.130457
34,34,0.029210
36,36,-0.090561
39,39,-0.081572
43,43,-0.159695
